In [ ]:
from __future__ import print_function
%matplotlib notebook
import pymp

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import patches
import copy
import math
import cubic_spline_planner
import warnings
from functools import reduce
import pickle
warnings.filterwarnings('ignore')
import time
from tqdm import tqdm_notebook as tqdm
SIM_LOOP = 150

# Parameter
MAX_SPEED = 2.26  # maximum speed [m/s]
MAX_LAT_ACCEL = 0.5  # maximum acceleration [m/ss]
MAX_LONG_ACCEL = 1.6  # maximum acceleration [m/ss]
MAX_CURVATURE = 1/1.0  # smaximum curvature [1/m]
MAX_ROAD_WIDTH = 0  # maximum road width [m]
D_ROAD_W = 1 # road width sampling length [m]
DT = 2 # time tick [s]
DT2 = 0.5
MAXT = 12  # max prediction time [m]
MINT = 4  # min prediction time [m]
TARGET_SPEED = 2.25  # target speed [m/s]

D_T_S = 0.75  # target speed sampling length [m/s]
N_S_SAMPLE = 3  # sampling number of target speed
ROBOT_RADIUS = 2.0  # robot radius [m]

# cost weights
KJ = 0.001
KT = 1.0
KD = 100
KLAT = 10.0
KLON = 1.0

show_animation = True

class quintic_polynomial:

    def __init__(self, xs, vxs, axs, xe, vxe, axe, T):
        # calc coefficient of quintic polynomial
        self.a0 = xs
        self.a1 = vxs
        self.a2 = axs / 2.0

        A = np.array([[T ** 3, T ** 4, T ** 5],
                      [3 * T ** 2, 4 * T ** 3, 5 * T ** 4],
                      [6 * T, 12 * T ** 2, 20 * T ** 3]])
        b = np.array([xe - self.a0 - self.a1 * T - self.a2 * T ** 2,
                      vxe - self.a1 - 2 * self.a2 * T,
                      axe - 2 * self.a2])
        x = np.linalg.solve(A, b)

        self.a3 = x[0]
        self.a4 = x[1]
        self.a5 = x[2]

    def calc_point(self, t):
        xt = self.a0 + self.a1 * t + self.a2 * t ** 2 + \
             self.a3 * t ** 3 + self.a4 * t ** 4 + self.a5 * t ** 5

        return xt

    def calc_first_derivative(self, t):
        xt = self.a1 + 2 * self.a2 * t + \
             3 * self.a3 * t ** 2 + 4 * self.a4 * t ** 3 + 5 * self.a5 * t ** 4

        return xt

    def calc_second_derivative(self, t):
        xt = 2 * self.a2 + 6 * self.a3 * t + 12 * self.a4 * t ** 2 + 20 * self.a5 * t ** 3

        return xt

    def calc_third_derivative(self, t):
        xt = 6 * self.a3 + 24 * self.a4 * t + 60 * self.a5 * t ** 2

        return xt


class quartic_polynomial:

    def __init__(self, xs, vxs, axs, vxe, axe, T):
        # calc coefficient of quartic polynomial

        self.a0 = xs
        self.a1 = vxs
        self.a2 = axs / 2.0

        A = np.array([[3 * T ** 2, 4 * T ** 3],
                      [6 * T, 12 * T ** 2]])
        b = np.array([vxe - self.a1 - 2 * self.a2 * T,
                      axe - 2 * self.a2])
        x = np.linalg.solve(A, b)

        self.a3 = x[0]
        self.a4 = x[1]

    def calc_point(self, t):
        xt = self.a0 + self.a1 * t + self.a2 * t ** 2 + \
             self.a3 * t ** 3 + self.a4 * t ** 4

        return xt

    def calc_first_derivative(self, t):
        xt = self.a1 + 2 * self.a2 * t + \
             3 * self.a3 * t ** 2 + 4 * self.a4 * t ** 3

        return xt

    def calc_second_derivative(self, t):
        xt = 2 * self.a2 + 6 * self.a3 * t + 12 * self.a4 * t ** 2

        return xt

    def calc_third_derivative(self, t):
        xt = 6 * self.a3 + 24 * self.a4 * t

        return xt


class Frenet_path:
    def __init__(self, t=None, d=None, d_d=None, d_dd=None, d_ddd=None, s=None, s_d=None, s_dd=None, s_ddd=None, cd=0,
                 cv=0, cf=0, x=None, y=None, yaw=None, ds=None, c=None):
        self.t = t or []
        self.d = d or []
        self.d_d = d_d or []
        self.d_dd = d_dd or []
        self.d_ddd = d_ddd or []
        self.s = s or []
        self.s_d = s_d or []
        self.s_dd = s_dd or []
        self.s_ddd = s_ddd or []
        self.cd = cd or 0
        self.cv = cv or 0
        self.cf = cf or 0

        self.x = x or []
        self.y = y or []
        self.yaw = yaw or []
        self.ds = ds or []
        self.c = c or []

    def __add__(self, o):
        return Frenet_path(
            self.t[:-1] + [e + self.t[-1] for e in o.t],
            self.d[:-1] + o.d,
            self.d_d[:-1] + o.d_d,
            self.d_dd[:-1] + o.d_dd,
            self.d_ddd[:-1] + o.d_ddd,
            self.s[:-1] + o.s,
            self.s_d[:-1] + o.s_d,
            self.s_dd[:-1] + o.s_dd,
            self.s_ddd[:-1] + o.s_ddd,
            self.cd + o.cd,
            self.cv + o.cv,
            self.cf + o.cf,

            self.x[:-1] + o.x,
            self.y[:-1] + o.y,
            self.yaw[:-1] + o.yaw,
            self.ds[:-1] + o.ds,
            self.c[:-1] + o.c)
    def stitch (self, o, ind_offset):
        return Frenet_path(
            self.t,
            Frenet_path._stitch(self.d, o.d, ind_offset),
            Frenet_path._stitch(self.d_d, o.d_d, ind_offset),
            Frenet_path._stitch(self.d_dd, o.d_dd, ind_offset),
            Frenet_path._stitch(self.d_ddd, o.d_ddd, ind_offset),
            Frenet_path._stitch(self.s, o.s, ind_offset),
            Frenet_path._stitch(self.s_d, o.s_d, ind_offset),
            Frenet_path._stitch(self.s_dd, o.s_dd, ind_offset),
            Frenet_path._stitch(self.s_ddd, o.s_ddd, ind_offset),
            self.cd,
            self.cv,
            self.cf,
            Frenet_path._stitch(self.x, o.x, ind_offset),
            Frenet_path._stitch(self.y, o.y, ind_offset),
            Frenet_path._stitch(self.yaw, o.yaw, ind_offset),
            Frenet_path._stitch(self.ds, o.ds, ind_offset),
            Frenet_path._stitch(self.c, o.c, ind_offset))


        
    def _stitch (new, old, ind_offset):
        old_overlap = old[ind_offset:]
        min_len = min(len(old_overlap), len(new))
        old_overlap = old_overlap[:min_len]
        new_overlap = new[:min_len]
        x = np.linspace(0, 1, min_len)
        y = 1 / (1 + np.exp(-x * 5 -2.5))
        tmp = new_overlap * y + old_overlap * (1-y)

        return list(tmp) + new[len(tmp):]



def calc_frenet_path(c_speed, c_acc, c_d, c_d_d, c_d_dd, s0, Ti, di, tv):
    fp = Frenet_path()
    lat_qp = quintic_polynomial(c_d, c_d_d, c_d_dd, di, 0.0, 0.0, Ti)

    fp.t = [t for t in np.linspace(0.0, Ti, round(Ti / DT2) + 1)]
    fp.d = [lat_qp.calc_point(t) for t in fp.t]
    fp.d_d = [lat_qp.calc_first_derivative(t) for t in fp.t]
    fp.d_dd = [lat_qp.calc_second_derivative(t) for t in fp.t]
    fp.d_ddd = [lat_qp.calc_third_derivative(t) for t in fp.t]

    lon_qp = quartic_polynomial(s0, c_speed, c_acc, tv, 0.0, Ti)
        

    fp.s = [lon_qp.calc_point(t) for t in fp.t]
    fp.s_d = [lon_qp.calc_first_derivative(t) for t in fp.t]
    fp.s_dd = [lon_qp.calc_second_derivative(t) for t in fp.t]
    fp.s_ddd = [lon_qp.calc_third_derivative(t) for t in fp.t]

    Jp = np.sum(np.power(fp.d_ddd, 2))  # square of jerk
    Js = np.sum(np.power(fp.s_ddd, 2))  # square of jerk

    # square of diff from target speed
    ds = (TARGET_SPEED - fp.s_d[-1]) ** 2

    fp.cd = KJ * Jp + KT * Ti + KD * fp.d[-1] ** 2
    fp.cv = KJ * Js + KT * Ti + KD * ds
    fp.cf = KLAT * fp.cd + KLON * fp.cv
        

    return fp


def calc_path_segment(t, d, v, c_speed, c_acc, c_d, c_d_d, c_d_dd, s0, ob, csp):
    fp = calc_frenet_path(c_speed, c_acc, c_d, c_d_d, c_d_dd, s0, t, d, v)
    calc_global_path(fp, csp)
    if not check_path(fp, ob):
        fp.cf = np.inf
    return fp


def calc_optimal_path(c_speed, c_acc, c_d, c_d_d, c_d_dd, s0, ob, csp):
    d_vec = np.linspace(-MAX_ROAD_WIDTH, MAX_ROAD_WIDTH, round(2 * MAX_ROAD_WIDTH / D_ROAD_W + 1))
#    v_vec = np.linspace(TARGET_SPEED - D_T_S * N_S_SAMPLE, TARGET_SPEED + D_T_S * N_S_SAMPLE, 2 * N_S_SAMPLE + 1)
    v_vec = np.linspace(0,2.25, 2 * N_S_SAMPLE + 1)

    dyn_prog = [[[{'cost': np.inf, 'path': [], 's': s0} for e1 in range(len(v_vec))] for e2 in range(len(d_vec))] for e3 in range(round(MAXT / DT + 1))]
    for d_ind, d in enumerate(d_vec):
        for v_ind, v in enumerate(v_vec):
            dyn_prog[0][d_ind][v_ind]['cost'] = 0


    
    for dyn_ind in range(round(MINT / DT), round(MAXT / DT + 1)):
        ex_array = pymp.shared.list(range(len(d_vec) * len(v_vec)))
        with pymp.Parallel(6) as p:
            for d2_ind in p.range(0, len(d_vec)):
                d2 = d_vec[d2_ind]
                for v2_ind, v2 in enumerate(v_vec):
                    min_cost = np.inf
                    optim_path = []
                    optim_s = 0
                    for t_ind in range(0, dyn_ind - round(MINT / DT) + 1):
                        if t_ind == 0:
                            fp = calc_path_segment(dyn_ind * DT, d2, v2, c_speed, c_acc, c_d, c_d_d,c_d_dd, s0, ob, csp)                            
                            new_cost = fp.cf
                            if new_cost < min_cost:
                                min_cost = new_cost
                                optim_path = [(dyn_ind * DT, 0, 0, d2, v2)]
                                optim_s = fp.s[-1]
                        else:
                            for d1_ind, d1 in enumerate(d_vec):
                                for v1_ind, v1 in enumerate(v_vec):
                                    dyn_s0 = dyn_prog[t_ind][d1_ind][v1_ind]['s']
                                    fp = calc_path_segment((dyn_ind - t_ind) * DT, d2, v2, v1, 0, d1, 0, 0, dyn_s0, ob, csp)
                                    new_cost = dyn_prog[t_ind][d1_ind][v1_ind]['cost'] + fp.cf
                                    if new_cost < min_cost:
                                        min_cost = new_cost
                                        optim_path = dyn_prog[t_ind][d1_ind][v1_ind]['path'] + [((dyn_ind - t_ind) * DT, d1, v1, d2, v2)]
                                        optim_s = fp.s[-1]

                    ex_array[v2_ind + d2_ind * len(v_vec)] = (min_cost, optim_path, optim_s)
                                
        for d2_ind in range(len(d_vec)):
            for v2_ind in range(len(v_vec)):
                dyn_prog[dyn_ind][d2_ind][v2_ind]['cost'] = ex_array[v2_ind + d2_ind * len(v_vec)][0]
                dyn_prog[dyn_ind][d2_ind][v2_ind]['path'] = ex_array[v2_ind + d2_ind * len(v_vec)][1]
                dyn_prog[dyn_ind][d2_ind][v2_ind]['s'] = ex_array[v2_ind + d2_ind * len(v_vec)][2]
        
    final_min_cost = np.inf
    best_path = None
    for d_ind, d in enumerate(d_vec):
        for v_ind, v in enumerate(v_vec):
            if dyn_prog[-1][d_ind][v_ind]['cost'] < final_min_cost:
                final_min_cost = dyn_prog[-1][d_ind][v_ind]['cost']
                best_path = dyn_prog[-1][d_ind][v_ind]['path']

    optim_seg = best_path[0]
    segments = [calc_path_segment(optim_seg[0], optim_seg[3], optim_seg[4], c_speed, c_acc, c_d, c_d_d,c_d_dd, s0, ob, csp)]
    s = segments[-1].s[-1]
    for optim_seg in best_path[1:]:
        segment = calc_path_segment(optim_seg[0], optim_seg[3], optim_seg[4], optim_seg[2], 0, optim_seg[1], 0, 0, s, ob, csp)
        segments.append(segment)
        s = segment.s[-1]
        
    return reduce((lambda x,y:x+y), segments)


def unwrap_angle(x):
    x = np.fmod(x + np.pi, 2 * np.pi)
    if x < 0:
        x += 2 * np.pi
    return x - np.pi

def calc_global_path(fp, csp):
    for i in range(len(fp.s)):
        ix, iy = csp.calc_position(fp.s[i])
        if ix is None:
            break
        iyaw = csp.calc_yaw(fp.s[i])
        di = fp.d[i]
        fx = ix + di * math.cos(iyaw + math.pi / 2.0)
        fy = iy + di * math.sin(iyaw + math.pi / 2.0)
        fp.x.append(fx)
        fp.y.append(fy)

    # calc yaw and ds
    prev_yaw = 0
    for i in range(len(fp.x) - 1):
        dx = fp.x[i + 1] - fp.x[i]
        dy = fp.y[i + 1] - fp.y[i]
        if abs(dx) > 0.001 or abs(dy) > 0.001:
            yaw = math.atan2(dy, dx)
            fp.yaw.append(yaw)
            prev_yaw = yaw
        else:
            fp.yaw.append(prev_yaw)
        fp.ds.append(math.hypot(dx, dy))

    fp.yaw.append(fp.yaw[-1])
    fp.ds.append(fp.ds[-1])

    # calc curvature
    for i in range(len(fp.yaw) - 1):
        #fp.c.append(csp.calc_curvature(fp.s[i]))
        if fp.ds[i] > 0.2:
            fp.c.append(unwrap_angle(fp.yaw[i + 1] - fp.yaw[i]) / fp.ds[i])
        else:
            fp.c.append(0)


def check_collision(fp, ob):
    for i in range(len(ob[:, 0])):
        d = [((ix - ob[i, 0]) ** 2 + (iy - ob[i, 1]) ** 2)
             for (ix, iy) in zip(fp.x, fp.y)]

        collision = any([di <= ROBOT_RADIUS ** 2 for di in d])

        if collision:
            return False

    return True


def check_path(fp, ob):
#     if any([v > MAX_SPEED for v in fp.s_d]):  # Max speed check
#         return False
#     if any([v < 0 for v in fp.s_d]):  # Max speed check
#         return False
#     elif any([abs(a) > MAX_LONG_ACCEL for a in fp.s_dd]):  # Max accel check
#         return False
#     elif any([abs(v * v * c) > MAX_LAT_ACCEL for (v, c) in zip(fp.s_d, fp.c)]):  # Max accel check
#         return False
#     elif any([abs(c) > MAX_CURVATURE for c in fp.c]):  # Max curvature check
#         return False
    if not check_collision(fp, ob):
        return False

    return True


def generate_target_course(x, y):
    csp = cubic_spline_planner.Spline2D(x, y)
    s = np.arange(0, csp.s[-1], 0.1)

    rx, ry, ryaw, rk = [], [], [], []
    for i_s in s:
        ix, iy = csp.calc_position(i_s)
        rx.append(ix)
        ry.append(iy)
        ryaw.append(csp.calc_yaw(i_s))
        rk.append(csp.calc_curvature(i_s))

    return rx, ry, ryaw, rk, csp


# way points
(wx, wy) = pickle.load(open('ref_line.pkl','rb'))

s_o = np.cumsum(np.hypot(np.diff(wx),np.diff(wy)))

wx_o = copy.deepcopy(wx)
wy_o = copy.deepcopy(wy)
wx = wx_o[::20]
wy = wy_o[::20]
# rad = 10
# wx = rad*np.cos(np.linspace(0, 2 * np.pi, 40))-rad
# wy = rad*np.sin(np.linspace(0, 2 * np.pi, 40))

# obstacle lists
ob = np.array([[37,45]])

tx, ty, tyaw, tc, csp = generate_target_course(wx, wy)

L = 1.5

# initial state
s = 0.0  # current course position
v = 0.0  # current speed [m/s]
c_v = v
a = 0  # current acceleration [m/ss]
x = wx[0]
y = wy[0]
psi = math.atan2(wy[1] - wy[0], wx[1] - wx[0])
δ = 0
δ_dot = 0
c_d = 0.0  # current lateral position [m]
c_d_d = 0.0  # current lateral speed [m/s]
c_d_dd = 0.0  # current latral acceleration [m/s]

area = 20.0  # animation area length [m]

SIM_TIME = 150
dt = DT2
traj_sampling_time = 2
prev_traj_samp_time = -traj_sampling_time
prev_anim_time = 0.0
anim_time = DT2
t = 0.0
frame_num = 0

if show_animation:
    # initialise the graph and settings
    fig = plt.figure()
    ax = fig.gca()
    fig.show()
    fig.canvas.draw()

prev_path = None
for i in tqdm(range(round(SIM_TIME / dt))):
    if (t >= prev_traj_samp_time + traj_sampling_time):
        new_path = calc_optimal_path(v, 0, c_d, c_d_d, c_d_dd, s, ob, csp)
        if new_path is not None:
            if prev_path is not None:
                #path = new_path.stitch(prev_path, round(traj_sampling_time / DT2))
                path = new_path
            else:
                path = new_path
            prev_path = copy.deepcopy(path)
            prev_traj_samp_time = t
    
    if path is not None: 
        if path.cf == np.inf:
            print("No feasible path!")
            break
        ind = min(math.floor((t-prev_traj_samp_time)/DT2),len(path.yaw)-1)
        psi = path.yaw[ind]
        a = (path.s_d[ind] - v) * 100
        

    v = path.s_d[ind]      
    x = path.x[ind]
    y = path.y[ind]
    t += dt

    c_d = path.d[ind]
    c_d_d = path.d_d[ind]
    c_d_dd = path.d_dd[ind]
    s = path.s[ind]

    if show_animation and t > prev_anim_time + anim_time:  # pragma: no cover
        prev_anim_time = t
        ax.clear()
        # for stopping simulation with the esc key.
        ax.set_aspect('equal', adjustable='box')
        ax.plot(wx, wy, '.')
        ax.plot(tx, ty)
        ax.plot(ob[:, 0], ob[:, 1], "xk")
        ax.plot(path.x, path.y, "-or")
        ax.plot(path.x[ind], path.y[ind], "-oy")
        
        #ax.plot(x, y, ".c")
        rect = patches.Rectangle((x,y),2.5,1,psi*180/np.pi,fill=True)
        ax.add_patch(rect)
        rect = patches.Rectangle((x,y),-2.5,1,psi*180/np.pi,fill=False)
        ax.add_patch(rect)
        rect = patches.Rectangle((x,y),2.5,-1,psi*180/np.pi,fill=True)
        ax.add_patch(rect)
        rect = patches.Rectangle((x,y),-2.5,-1,psi*180/np.pi,fill=False)
        ax.add_patch(rect)

        plt.xlim(x - area, x + area)
        plt.ylim(y - area, y + area)
        plt.title("v[m/s]:" + '%.2f' % v + " s[m]:" + '%.2f' % s +' t[s]:' + '%.2f' % t)
        ax.grid(True)
        fig.canvas.draw()  # draw
        fig.savefig(str(frame_num)+'.jpg')
        frame_num +=1 
    time.sleep(0.001)  # sleep

print("Finish")